In [1]:
pip install transformers

  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=cf9bbf83d3b17f1f745924aef7e20641ba787c03b4563869831cf7fd376d83df
  Stored in directory: C:\Users\Ali\AppData\Local\pip\Cache\wheels\29\3c\fd\7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Note: you may need to restart the kernel to use updated packages.


# Transformers provides thousands of pretrained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, text generation, etc in 100+ languages. Its aim is to make cutting-edge NLP easier to use for everyone.

In [1]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

# To immediately use a model on a given text, we provide the pipeline API. Pipelines group together a pretrained model with the preprocessing that was used during that model training.

In [2]:
#huggingface summarization pipeline
summarizer = pipeline("summarization")

All model checkpoint layers were used when initializing TFT5Model.

All the layers of TFT5Model were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5Model for predictions without further training.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [3]:
#get data
#URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
URL = "https://towardsdatascience.com/rosy-ai-5897b1e1b5be"

In [4]:
r = requests.get(URL)

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [6]:
ARTICLE

"Sign in AI Art: Rosy AI Yariv Adan 4 days ago·4 min read In this work, we took a small detour from our main journey, in favour of a short colourful moment of reflection. We kept true to our mission, and used “AI” to generate digital art work, though not in the expected sense. Inspiration It was the summer of 2020, we were discussing new AI & Art projects ideas, and the world around us was spiralling. The dissonance was impossible to ignore. Inside our confined walls, we were surrounded by the vast and fast growing world of AI. Billions of dollars worth of datasets and TPU years, as well thousands of human years captured in research papers. All in the name of an implicit or explicit promise for a better future in science, health, agriculture, energy management, transportation, information consumption, human machine interaction, etc. Yet, it all seemed so disconnected from the reality of the world outside. These marvels of science and technology, these machine olympics — were mesmerisin

In [7]:
max_chunk = 500

In [8]:
ARTICLE = ARTICLE.replace('.','.<eos>')
ARTICLE = ARTICLE.replace('?','?<eos>')
ARTICLE = ARTICLE.replace('!','!<eos>')

In [9]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])


0


In [10]:
len(chunks)

3

In [12]:
summ = summarizer(chunks, max_length=160, min_length=70, do_sample=False)

In [13]:
summ[0]

{'summary_text': 'we wanted to capture this feeling in a piece of AI art . despite all that progress, as a society, we seem to end up at the same place . it was important for us to capture that tension between ingenuity in the details, and big picture blindness . but we didn’t want it to be ML in the regular sense .'}

In [14]:
text = ' '.join([res['summary_text'] for res in summ])

In [16]:
#save the output to text file
with open('Summary.txt', 'w') as f:    
    f.write(text)